# Proxy

It provides an interface to another object, acting as an intermediary or surrogate. It is used when we want to control access to an object, add some functionality before or after accessing it, or delay the creation of an object until it is actually needed.

he main parts:

 - Proxy Interface: The Proxy Interface defines the methods that both the Real Subject and the Proxy must implement. This ensures that the Proxy is a true substitute for the Real Subject.
 - Real Subject: The Real Subject is the actual object that the Proxy represents. It implements the Proxy Interface and performs the real business logic.
 - Proxy: The Proxy is the intermediary between the client and the Real Subject. It also implements the Proxy Interface, but it doesn’t perform the core logic itself. Instead, it delegates the calls to the Real Subject, potentially adding additional functionality before or after the delegation.

## Virtual Proxy

A Virtual Proxy is all about deferring a resource-intensive object loading until explicitly required (lazy instantiation). It creates a placeholder for an expensive or resource-intensive object. This object is only instantiated when a client requests it. Until then, the proxy manages the access.

In [2]:
# lagacy part
# We suppose this can't be modified

from abc import abstractmethod

class Image:
    @abstractmethod
    def draw(self):
        pass

class Bitmap(Image):
    def __init__(self, filename):
        self.filename = filename
        # load image from filename
        print(f"Loaded image {filename}")

    def draw(self):
        # draw the loaded image
        print(f"Draw image {self.filename}")

In [10]:
# for certain reasons such as :
# - lazy instantiation: instantiate Bitmap only when drawing
# - testing purpose where we don't want to load the actual image
# we don't want to do the image loading when instantiating the Bitmap object.
# So we can define a virtual class.

class VirtualBitmap(Image):
    
    def __init__(self, filename):
        self.filename = filename
        self.bmp = None

    def draw(self):
        if self.bmp is None:
            self.bmp = Bitmap(self.filename)
        self.bmp.draw()

In [11]:
class Client:
    @staticmethod
    def draw_image(img:Image):
        img.draw()

bmp = VirtualBitmap("myimage.bmp") # placeholder
Client.draw_image(bmp) # actual loading

Loaded image myimage.bmp
Draw image myimage.bmp


# Communication proxy

Also called 'over the wire'. Relays communication with objects residing in different address spaces or on remote servers. The proxy handles communication details, such as network connections, serialization, and deserialization, while the client interacts with the proxy as if it were the real object.

In [2]:
# interface for both local and remote services

from abc import abstractmethod

class Service:
    @abstractmethod
    def request(self, msg):
        raise NotImplementedError

In [14]:
# remote service
class RemoteService(Service):
    def request(self, msg):
        return "received: " + msg

In [15]:
# proxy service
class ProxyService(Service):
    def __init__(self, service):
        self.service = service
    def request(self, msg):
        msg = "sending " + msg
        response = self.service.request(msg)
        response = "remote " + response
        return response

In [16]:
rs = RemoteService()
my_service = ProxyService(rs)
my_service.request("Send a test")

'remote received: sending Send a test'

## Caching Proxy

The use cases can be:
 - buffering: buffer recieved informations with lagged processing
 - caching: cache tasks for asychronesous processing
 - expaced sending: not to overfloat query server

In [16]:
import time

# Define the interface for the Real Subject
class DatabaseQuery:
    def execute_query(self, query):
        pass

# Real Subject: Represents the actual database
class RealDatabaseQuery(DatabaseQuery):
    def execute_query(self, query):
        print(f"Executing query: {query}")
        # Simulate a database query and return the results
        return f"Results for query: {query}"

# Proxy: Caching Proxy for Database Queries
class CacheProxy(DatabaseQuery):
    def __init__(self, real_database_query, cache_duration_seconds):
        self._real_database_query = real_database_query
        self._cache = {}
        self._cache_duration = cache_duration_seconds

    def execute_query(self, query):
        if query in self._cache and time.time() - self._cache[query]["timestamp"] <= self._cache_duration:
            # Return cached result if it's still valid
            print(f"CacheProxy: Returning cached result for query: {query}")
            return self._cache[query]["result"]
        else:
            # Execute the query and cache the result
            result = self._real_database_query.execute_query(query)
            self._cache[query] = {"result": result, "timestamp": time.time()}
            return result


In [17]:
# Client code
# Create the Real Subject
real_database_query = RealDatabaseQuery()

# Create the Cache Proxy with a cache duration of 5 seconds
cache_proxy = CacheProxy(real_database_query, cache_duration_seconds=5)

# Perform database queries, some of which will be cached
print(cache_proxy.execute_query("SELECT * FROM table1"))
print(cache_proxy.execute_query("SELECT * FROM table2"))
time.sleep(3)  # Sleep for 3 seconds

# Should return cached result
print(cache_proxy.execute_query("SELECT * FROM table1"))

print(cache_proxy.execute_query("SELECT * FROM table3"))

Executing query: SELECT * FROM table1
Results for query: SELECT * FROM table1
Executing query: SELECT * FROM table2
Results for query: SELECT * FROM table2
CacheProxy: Returning cached result for query: SELECT * FROM table1
Results for query: SELECT * FROM table1
Executing query: SELECT * FROM table3
Results for query: SELECT * FROM table3
